In [3]:
import pandas as pd
import numpy as np

In [4]:
male_df = pd.read_csv("C:/Users/JEONGHEE/Desktop/당뇨병플젝/male.csv")
female_df = pd.read_csv("C:/Users/JEONGHEE/Desktop/당뇨병플젝/female.csv")

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.utils import resample
import statsmodels.api as sm


basic_drop_cols = ['DS1_ID', 'DS1_SEX', 'DS1_GLU0', 'DS1_HBA1C',
             'DS1_WALK', 'DS1_WALKT', 'DS1_WAIST', 'DS1_BODYFAT', 'DS1_MARRY_RE'] 
X = male_df.drop(columns=basic_drop_cols + ['target'])
y = male_df['target']


df_all = X.copy()
df_all['target'] = y
df_majority = df_all[df_all['target'] == 0]
df_minority = df_all[df_all['target'] == 1]
df_majority_down = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)
df_balanced = pd.concat([df_majority_down, df_minority])
X_bal = df_balanced.drop(columns=['target'])
y_bal = df_balanced['target']


scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_bal), columns=X_bal.columns, index=X_bal.index)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_bal, test_size=0.2, stratify=y_bal, random_state=42
)


X_train_const = sm.add_constant(X_train)
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit()


print(result.summary())

X_test_const = sm.add_constant(X_test)
X_test_const = X_test_const[X_train_const.columns]  # ensure same columns order

y_pred_prob = result.predict(X_test_const)
y_pred_label = (y_pred_prob >= 0.5).astype(int)

print("\nClassification Report (Test):")
print(classification_report(y_test, y_pred_label))
print("AUC:", round(roc_auc_score(y_test, y_pred_prob), 3))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_label))

Optimization terminated successfully.
         Current function value: 0.628057
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                 2571
Model:                          Logit   Df Residuals:                     2551
Method:                           MLE   Df Model:                           19
Date:                Mon, 21 Apr 2025   Pseudo R-squ.:                 0.09390
Time:                        01:58:29   Log-Likelihood:                -1614.7
converged:                       True   LL-Null:                       -1782.1
Covariance Type:            nonrobust   LLR p-value:                 1.476e-59
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.0015      0.042     -0.036      0.971      -0.084       0.081
DS1_AGE           0.

In [6]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.utils import resample
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor


basic_drop_cols = ['DS1_ID', 'DS1_SEX', 'DS1_GLU0', 'DS1_HBA1C',
             'DS1_WALK', 'DS1_WALKT', 'DS1_WAIST', 'DS1_BODYFAT', 'DS1_MARRY_RE'] 
X = male_df.drop(columns=basic_drop_cols + ['target'])
y = male_df['target']

# 언더샘플링
df_all = X.copy()
df_all['target'] = y
df_majority = df_all[df_all['target'] == 0]
df_minority = df_all[df_all['target'] == 1]
df_majority_down = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)
df_balanced = pd.concat([df_majority_down, df_minority])
X_bal = df_balanced.drop(columns=['target'])
y_bal = df_balanced['target']


scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_bal), columns=X_bal.columns, index=X_bal.index)


def remove_high_vif(X, thresh=10.0):
    variables = X.columns.tolist()
    while True:
        vif = [variance_inflation_factor(X[variables].values, i) for i in range(len(variables))]
        max_vif = max(vif)
        if max_vif > thresh:
            max_index = vif.index(max_vif)
            print(f"제거된 변수: {variables[max_index]} (VIF: {max_vif:.2f})")
            variables.pop(max_index)
        else:
            break
    return X[variables]

X_vif = remove_high_vif(X_scaled, thresh=10.0)


X_train, X_test, y_train, y_test = train_test_split(
    X_vif, y_bal, test_size=0.2, stratify=y_bal, random_state=42
)


X_train_const = sm.add_constant(X_train)
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit()
print(result.summary())

X_test_const = sm.add_constant(X_test)
X_test_const = X_test_const[X_train_const.columns]  # ensure same columns order

y_pred_prob = result.predict(X_test_const)
y_pred_label = (y_pred_prob >= 0.5).astype(int)

print("\nClassification Report (Test):")
print(classification_report(y_test, y_pred_label))
print("AUC:", round(roc_auc_score(y_test, y_pred_prob), 3))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_label))


Optimization terminated successfully.
         Current function value: 0.628057
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                 2571
Model:                          Logit   Df Residuals:                     2551
Method:                           MLE   Df Model:                           19
Date:                Mon, 21 Apr 2025   Pseudo R-squ.:                 0.09390
Time:                        01:58:30   Log-Likelihood:                -1614.7
converged:                       True   LL-Null:                       -1782.1
Covariance Type:            nonrobust   LLR p-value:                 1.476e-59
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.0015      0.042     -0.036      0.971      -0.084       0.081
DS1_AGE           0.

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.utils import resample
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from imblearn.under_sampling import RandomUnderSampler


basic_drop_cols = ['DS1_ID', 'DS1_SEX', 'DS1_GLU0', 'DS1_HBA1C',
             'DS1_WALK', 'DS1_WALKT', 'DS1_WAIST', 'DS1_BODYFAT', 'DS1_MARRY_RE'] 
X = male_df.drop(columns=basic_drop_cols + ['target'])
y = male_df['target']


undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X, y)

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_resampled), columns=X.columns, index=X_resampled.index)


def remove_high_vif(X, thresh=5.0):
    variables = X.columns.tolist()
    while True:
        vif = [variance_inflation_factor(X[variables].values, i) for i in range(len(variables))]
        max_vif = max(vif)
        if max_vif > thresh:
            max_index = vif.index(max_vif)
            print(f"제거된 변수: {variables[max_index]} (VIF: {max_vif:.2f})")
            variables.pop(max_index)
        else:
            break
    return X[variables]

X_vif = remove_high_vif(X_scaled, thresh=5.0)


X_train, X_test, y_train, y_test = train_test_split(
    X_vif, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42
)

X_train_const = sm.add_constant(X_train)
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit()

print(result.summary())

X_test_const = sm.add_constant(X_test)
X_test_const = X_test_const[X_train_const.columns]  # ensure same columns order

y_pred_prob = result.predict(X_test_const)
y_pred_label = (y_pred_prob >= 0.5).astype(int)

print("\nClassification Report (Test):")
print(classification_report(y_test, y_pred_label))
print("AUC:", round(roc_auc_score(y_test, y_pred_prob), 3))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_label))

제거된 변수: DS1_PBF (VIF: 6.86)
제거된 변수: DS1_BMI (VIF: 5.13)
Optimization terminated successfully.
         Current function value: 0.628538
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                 2571
Model:                          Logit   Df Residuals:                     2553
Method:                           MLE   Df Model:                           17
Date:                Mon, 21 Apr 2025   Pseudo R-squ.:                 0.09321
Time:                        02:10:25   Log-Likelihood:                -1616.0
converged:                       True   LL-Null:                       -1782.1
Covariance Type:            nonrobust   LLR p-value:                 2.428e-60
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0014      0.042      0.033

In [11]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor


basic_drop_cols = ['DS1_ID', 'DS1_SEX', 'DS1_GLU0', 'DS1_HBA1C',
             'DS1_WALK', 'DS1_WALKT', 'DS1_WAIST', 'DS1_BODYFAT', 'DS1_MARRY_RE'] 
X = male_df.drop(columns=basic_drop_cols + ['target'])
y = male_df['target']


undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X, y)


scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_resampled), columns=X.columns, index=X_resampled.index)


def remove_high_vif(X, thresh=5.0):
    variables = X.columns.tolist()
    while True:
        vif = [variance_inflation_factor(X[variables].values, i) for i in range(len(variables))]
        max_vif = max(vif)
        if max_vif > thresh:
            max_index = vif.index(max_vif)
            print(f"제거된 변수: {variables[max_index]} (VIF: {max_vif:.2f})")
            variables.pop(max_index)
        else:
            break
    return X[variables]

X_vif = remove_high_vif(X_scaled, thresh=5.0)


X_train, X_test, y_train, y_test = train_test_split(
    X_vif, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42
)


def fit_logit_and_filter(X, y, p_thresh=0.05):
    X_const = sm.add_constant(X)
    model = sm.Logit(y, X_const).fit(disp=0)
    while True:
        pvalues = model.pvalues.drop('const', errors='ignore')
        max_pval = pvalues.max()
        if max_pval > p_thresh:
            worst_feature = pvalues.idxmax()
            print(f"제거된 변수 (p > {p_thresh}): {worst_feature} ({max_pval:.4f})")
            X = X.drop(columns=[worst_feature])
            X_const = sm.add_constant(X)
            model = sm.Logit(y, X_const).fit(disp=0)
        else:
            break
    return model, X

result, X_train_final = fit_logit_and_filter(X_train, y_train)


print(result.summary())


X_test_final = X_test[X_train_final.columns]
X_test_const = sm.add_constant(X_test_final)

y_pred_prob = result.predict(X_test_const)
y_pred_label = (y_pred_prob >= 0.5).astype(int)

print("\nClassification Report (Test):")
print(classification_report(y_test, y_pred_label))
print("AUC:", round(roc_auc_score(y_test, y_pred_prob), 3))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_label))

제거된 변수: DS1_PBF (VIF: 6.86)
제거된 변수: DS1_BMI (VIF: 5.13)
제거된 변수 (p > 0.05): DS1_DRINK_RE (0.6089)
제거된 변수 (p > 0.05): DS1_HIP (0.3774)
제거된 변수 (p > 0.05): DS1_EXER (0.3799)
제거된 변수 (p > 0.05): total_exer (0.6313)
제거된 변수 (p > 0.05): DS1_DBP (0.3682)
제거된 변수 (p > 0.05): DS1_HTN (0.1716)
제거된 변수 (p > 0.05): DS1_INCOME_RE (0.1481)
제거된 변수 (p > 0.05): DS1_EDU_RE (0.1280)
제거된 변수 (p > 0.05): DS1_WALKFQ (0.0727)
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                 2571
Model:                          Logit   Df Residuals:                     2562
Method:                           MLE   Df Model:                            8
Date:                Mon, 21 Apr 2025   Pseudo R-squ.:                 0.08974
Time:                        02:10:35   Log-Likelihood:                -1622.2
converged:                       True   LL-Null:                       -1782.1
Covariance Type:            nonrobust   LLR p-